In [1]:
#!git clone https://github.com/elina22-hub/11785_SceneDetection_Quantization.git

In [2]:
#!pip install onnxruntime

In [3]:
# copy data to 11785 directory
#%cp unique-142p.zip 11785_SceneDetection_Quantization/

In [129]:
# copy scence chagne.csv to 11785 directory
#%cp scene-change.csv 11785_SceneDetection_Quantization/

/bin/bash: switchml: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `switchml'
/bin/bash: _moduleraw: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `_moduleraw'


In [142]:
# set current path to github model path
%cd ./11785_SceneDetection_Quantization

/home/ubuntu/11785_SceneDetection_Quantization


In [101]:
#!unzip \*.zip

In [143]:
%pwd

'/home/ubuntu/11785_SceneDetection_Quantization'

In [144]:
# packages for Quantization
import os
import yaml
import onnx
import onnx.numpy_helper
import onnxruntime
import onnxruntime.quantization

# packages for loading the model
from model.beta_vae import *
import torch.utils.data 
from torchvision import datasets, transforms
import pdb
import pandas as pd
import torch.nn.functional as F
import torch
import numpy as np
from scene_detection import *
import torchvision.utils as vutils
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import shutil
import warnings

warnings.filterwarnings("ignore")

cuda = torch.cuda.is_available()
NUM_WORKERS = 4 if cuda else 0
DEVICE = torch.device("cuda" if cuda else "cpu")
BATCH_SIZE = 64
IMG_SIZE = 64

print(torch.__version__)
print(cuda)
print(NUM_WORKERS)
print(DEVICE)

1.8.1+cu111
False
0
cpu


In [6]:
# model hyperparameters (beta = 2)
with open("config.yaml", "r") as file:
    config = yaml.safe_load(file)

In [7]:
config

{'model_params': {'name': 'BetaVAE',
  'beta': 2,
  'in_channels': 3,
  'latent_dim': 10,
  'loss_type': 'H',
  'max_capacity': 25,
  'Capacity_max_iter': 10000,
  'image_height': 64,
  'image_width': 64,
  'hidden_dims': [32, 64, 128, 256, 512]},
 'exp_params': {'img_size': 64,
  'train_data_dir': './data_train',
  'val_data_dir': './data_val',
  'model_save_dir': './best_model',
  'batch_size': 256,
  'learning_rate': 0.002,
  'weight_decay': 0.0,
  'scheduler_gamma': 0.6,
  'max_epochs': 250,
  'manual_seed': 1265,
  'log_save_dir': './logs',
  'log_save_name': 'BetaVAE',
  'pic_save_dir': './scene_divergence_graph'}}

In [8]:
# load model
model = BetaVAE(**config['model_params'])
model_name = "Model_baseline"
model_path = "./best_model/" + model_name
checkpoint = torch.load(model_path, map_location="cpu")
model.load_state_dict(checkpoint['model_state_dict'])
# make sure setting the model to eval mode before exporting to ONNX
model.eval()

BetaVAE(
  (encoder): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): ReLU()
    )
    (1): Sequential(
      (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): ReLU()
    )
    (2): Sequential(
      (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): ReLU()
    )
    (3): Sequential(
      (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): ReLU()
    )
    (4): Sequential(
      (0): Conv2d(256, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): ReLU()
    )
  )
  (fc_mu): Linear(in_features=2048, out_features=10, bias=True)
  (fc_var): Linear(in_features=2048, out_features=10, bias=True)
  (decoder_input): Linear(in_features=10, out_features=2048, bias=True)
  (decoder): Sequential(
    (0): Sequential(
      (0): ConvTranspose2d(512, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), output_paddin

### Data Processing (Run Only Once Unless Terminal Crash)

Notice: Unique images index is not continuous. Also, .DS_Store is in the imagefile, remove it

In [311]:
%pwd

'/home/ubuntu/11785_SceneDetection_Quantization'

In [308]:
# pick the validation set from unique 142p dataset
imagefile = os.listdir("unique-142p")
imageset = set(imagefile)
old_path = "./unique-142p"
new_path = "./unique-142p-valid/142p-image/"
data_dir = "./unique-142p-valid/"
new_data_dir = "./unique-142p-valid/142p-image/"

In [309]:
# pick validation images
def get_valid_image_list(imagefile):
    """
    input: imagefile -> the unique-142p file
    return: valid_image_list -> image name list for validation
    """
    valid_image_list = []
    st_idx = 1
    ed_idx = 25947
    for image in imagefile:
        if image != ".DS_Store":
            idx = int(image[6:12])
            if st_idx <= idx <= ed_idx:
                valid_image_list.append(image)
    return valid_image_list

In [310]:
# create file and put it under another file name
def copy_valid_image(valid_image_list, old_path, new_path):
    # determine whether new path exists
    if not os.path.exists(new_path):
        os.makedirs(new_path)
    for valid_image in valid_image_list:
        shutil.copyfile(os.path.join(old_path, valid_image), os.path.join(new_path, valid_image))

In [312]:
# get validation image list and put it under the corresponding directory
valid_image_list = get_valid_image_list(imagefile)
copy_valid_image(valid_image_list, old_path, new_path)

In [316]:
# create subfolder for each images in the folder, with name of the subfolder being the int(image[6:12]) name
def create_imagefolder(valid_image_list, image_path):
    for image in os.listdir(image_path):
        os.makedirs(os.path.join(image_path, "folder_"+image))
    # put each image into the subfolder
    for valid_image in valid_image_list:
        shutil.move(os.path.join(image_path, valid_image), os.path.join(image_path, "folder_"+valid_image))

In [317]:
# create subfolder for each images in the folder, with name of the subfolder being the int(image[6:12]) name
create_imagefolder(valid_image_list, new_path)

In [335]:
%cd unique-142p-valid/142p-image/

/home/ubuntu/11785_SceneDetection_Quantization/unique-142p-valid/142p-image


In [320]:
%ls

/bin/bash: switchml: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `switchml'
/bin/bash: _moduleraw: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `_moduleraw'
folder_frame_000001.png/  folder_frame_006411.png/  folder_frame_016038.png/
folder_frame_000012.png/  folder_frame_006412.png/  folder_frame_016039.png/
folder_frame_000015.png/  folder_frame_006413.png/  folder_frame_016040.png/
folder_frame_000016.png/  folder_frame_006414.png/  folder_frame_016041.png/
folder_frame_000017.png/  folder_frame_006415.png/  folder_frame_016042.png/
folder_frame_000018.png/  folder_frame_006416.png/  folder_frame_016043.png/
folder_frame_000020.png/  folder_frame_006417.png/  folder_frame_016044.png/
folder_frame_000021.png/  folder_frame_006418.png/  folder_frame_016045.png/
folder_frame_000022.png/  folder_frame_006420.png/  folder_frame_016046.png/
folder_frame_000023.png/  folder_frame_006424.p

In [339]:
%cd ../

/home/ubuntu/11785_SceneDetection_Quantization


In [340]:
%pwd

'/home/ubuntu/11785_SceneDetection_Quantization'

In [220]:
%mkdir unique-142p-valid-backup

/bin/bash: switchml: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `switchml'
/bin/bash: _moduleraw: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `_moduleraw'


In [336]:
# create reference index
ref_image_folder_indices = os.listdir()
ref_image_folder_indices.sort()
print(ref_image_folder_indices[:10])

['folder_frame_000001.png', 'folder_frame_000012.png', 'folder_frame_000015.png', 'folder_frame_000016.png', 'folder_frame_000017.png', 'folder_frame_000018.png', 'folder_frame_000020.png', 'folder_frame_000021.png', 'folder_frame_000022.png', 'folder_frame_000023.png']


### DataLoader

In [341]:
def load_data(phase, data_dir):
    transform_list = data_transforms(phase)
    if phase == "train":
        dataset = datasets.ImageFolder(data_dir, transform=transform_list)
        data_loader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True, num_workers=NUM_WORKERS)
    else:
        dataset = datasets.ImageFolder(data_dir, transform=transform_list)
        data_loader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True, num_workers=NUM_WORKERS)
    return dataset, data_loader
        
def data_transforms(phase):
    if phase == "train":
        transform = transforms.Compose([transforms.Resize((IMG_SIZE, IMG_SIZE)),
                                        transforms.RandomHorizontalFlip(p=0.5),
                                        transforms.ColorJitter(brightness=0.8, contrast=0.8, saturation=0.8),
                                        transforms.ToTensor()])
    else:
        transform = transforms.Compose([transforms.Resize((IMG_SIZE, IMG_SIZE)),
                                        transforms.ToTensor()])
    return transform

In [342]:
# create validation dataloader
valid_dataset, valid_dataloader = load_data(phase="eval", data_dir=new_data_dir)
example_valid_input = next(iter(valid_dataloader))[0]
print("\nDetails of the validation dataset and data loader are below.\n")
print("Num Observations: {}\nNum Batches:\t  {}\nBatch Shape:\t  {}\nNum Classes:\t  {}".\
      format(len(valid_dataset), len(valid_dataloader), list(example_valid_input.shape), len(valid_dataset.classes)))
print("True Observations used: ", BATCH_SIZE * len(valid_dataloader))


Details of the validation dataset and data loader are below.

Num Observations: 14196
Num Batches:	  221
Batch Shape:	  [64, 3, 64, 64]
Num Classes:	  14196
True Observations used:  14144


In [343]:
# create dictionary for matching
batch_idx = [i for i in range(BATCH_SIZE * len(valid_dataloader))]
batch_to_image = dict(zip(batch_idx, ref_image_folder_indices))

### Convert Pytorch model to ONNX model

In [24]:
# model path
onnx_model_file = "./onnx_model/"

In [25]:
if not os.path.exists(onnx_model_file):
    os.mkdir(onnx_model_file)

In [29]:
# convert pytorch model to ONNX model
onnx_model_path = onnx_model_file + model_name + ".onnx"     # float 32
qi_onnx_model_path = onnx_model_file + "QI-" + model_name + ".onnx"  # int 8
print(onnx_model_path)
print(qi_onnx_model_path)

./onnx_model/Model_baseline.onnx
./onnx_model/QI-Model_baseline.onnx


In [40]:
# input to the model (same shape as batch input shape), just put some random input in
x = torch.ones(64, 3, 64, 64)
# Export pytorch model as onnx model
torch.onnx.export(model, x, onnx_model_path, input_names=["input"], output_names=["output"], opset_version=11)

In [32]:
#!pip install --upgrade onnx

In [41]:
# load onnx model
onnx_model = onnx.load(onnx_model_path)
onnx.checker.check_model(onnx_model)

### Quantize Float32 Model to Int8 Model

In [62]:
# per channel=True???
kwargs = {"op_types_to_quantize":     [],
          "per_channel":              True,
          "reduce_range":             False,
          "activation_type":          onnxruntime.quantization.QuantType.QUInt8,
          "weight_type":              onnxruntime.quantization.QuantType.QUInt8,
          "nodes_to_quantize":        [],
          "nodes_to_exclude":         [],
          "use_external_data_format": False}

onnxruntime.quantization.quantize_dynamic(onnx_model_path, qi_onnx_model_path, **kwargs)

In [63]:
# Compare model parameters size
onnx_size_mb   = os.path.getsize(onnx_model_path)/1e6
qi_onnx_size_mb = os.path.getsize(qi_onnx_model_path)/1e6

print("\nONNX model size (MB):     {:.2f}".format(onnx_size_mb))
print("\nQI-ONNX model size (MB):  {:.2f}".format(qi_onnx_size_mb))


ONNX model size (MB):     12.84

QI-ONNX model size (MB):  7.96


### Perform Inference on Input Images

In [379]:
class init_inference:
    def __init__(self, session):
        self.session = session
        
    def __call__(self, image):
        input_data = image.numpy()
        input_name  = self.session.get_inputs()[0].name
    
        return self.session.run(None, {input_name: input_data})

def mean_latency_fn(tqdm_dataloader):
    start = tqdm_dataloader.start_t
    stop = tqdm_dataloader.last_print_t
    n = tqdm_dataloader.n
    mean_latency = (stop - start)/n
    return mean_latency

def simple_table(data_multilist, x_label_list, y_label_list):
    for i in range(len(y_label_list)+1):
        if i == 0:
            row = ["\t"]
            row.extend(x_label_list)
            string = "\t".join(row)
        else:
            row = [y_label_list[i-1]]
            
            row.extend([format(x, ".6f") for x in data_multilist[i-1]])
            string = "\t".join(row)
            
        print(string)
        
    return None

def plot_statistic(x, y, title, x_label, y_label):

    label_pos = [i for i, _ in enumerate(x)]

    plt.figure(figsize=(12, 6))
    plt.bar(label_pos, x, color='black', width=0.8)
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.title(title)
    plt.xticks(label_pos, y)
    
    plt.show()
    
    return None

def valid_model(valid_dataloader, session):
    tqdm_dataloader = tqdm(valid_dataloader)
    inference = init_inference(session)
    valid_loss = {'val_total_loss':0.0, 'val_recon_loss':0.0,'val_kld':0.0}
    len_data_loader = len(valid_dataloader)
    
    # Latent Space
    mu = None
    logvar = None
    
    # origin_images
    origin_images_list = []
    index_list = []
    
    for _, (images, y) in enumerate(tqdm_dataloader):
        results = inference(images)
        # covert results into tensor
        results_converted = []
        for result in results:
            result = torch.tensor(result)
            results_converted.append(result)
        
        # Latent Space
        latent_u = results_converted[2]
        latent_logvar = results_converted[3]
        
        loss = model.loss_function(*results_converted,M_N = 1.0/float(len_data_loader), batch_size=BATCH_SIZE)
        
        valid_loss['val_total_loss'] += loss['total_loss'].cpu().detach().numpy()
        valid_loss['val_recon_loss'] += loss['recon_loss'].cpu().detach().numpy()
        valid_loss['val_kld'] += loss['kld'].cpu().detach().numpy()
        
        # Latent Space
        if mu == None and logvar == None:
            mu = latent_u
            logvar = latent_logvar
        else:
            mu = torch.cat((mu, latent_u))
            logvar = torch.cat((logvar, latent_logvar))
            
        # origin_images
        origin_images_list.append(images)
        index_list.append(y)
            
    valid_loss['val_total_loss'] /= len_data_loader
    valid_loss['val_recon_loss'] /= len_data_loader
    valid_loss['val_kld'] /= len_data_loader
    mean_latency = mean_latency_fn(tqdm_dataloader)

    origin_images_tensor = torch.cat(origin_images_list, dim=0)
    index_tensor = torch.cat(index_list, dim=0)
    
    return valid_loss, mean_latency, origin_images_tensor, index_tensor, mu.cpu().detach().numpy(), logvar.cpu().detach().numpy()

In [380]:
ort_session = onnxruntime.InferenceSession(onnx_model_path)
loss, mean_latency, origin_images_tensor, index_tensor, mu, logvar = valid_model(valid_dataloader, ort_session)

  0%|          | 0/221 [00:00<?, ?it/s]

In [381]:
print("Basline Model Loss (Float 32): ")
print(loss)

Basline Model Loss (Float 32): 
{'val_total_loss': 185.88345619132616, 'val_recon_loss': 140.68147370178775, 'val_kld': 22.6009912318234}


In [382]:
print(mean_latency)

0.22942406667303716


In [383]:
qi_ort_session = onnxruntime.InferenceSession(qi_onnx_model_path)
qi_loss, qi_mean_latency, qi_origin_images_tensor, qi_index_tensor, qi_mu, qi_logvar = valid_model(valid_dataloader, qi_ort_session)

  0%|          | 0/221 [00:00<?, ?it/s]

In [384]:
print("Quantized Basline Model Loss (Int 8): ")
print(qi_loss)

Quantized Basline Model Loss (Int 8): 
{'val_total_loss': 187.2001666763789, 'val_recon_loss': 141.90202206400184, 'val_kld': 22.649072112001445}


In [385]:
print(qi_mean_latency)

0.26821120939643134


In [386]:
print(qi_mean_latency - mean_latency)

0.038787142723394175


### Compare Accuracy of the scene change (Float 32 vs Int 8)

In [387]:
def KL_latent(mu1, logvar1, mu2, logvar2):
    latent_dim = mu1.shape[0]
    sigma1 = np.exp(0.5*logvar1)
    sigma2 = np.exp(0.5*logvar2)
    KL = 0.5*((sigma1 / sigma2).sum() + ((mu2 - mu1) * (mu2 - mu1) / sigma2).sum() - latent_dim + np.log(sigma2.prod() / sigma1.prod()))
    return KL

In [388]:
def compute_frame_KL_div(mu, logvar):
    """
    mu: 
    logvar:
    return: 
    """
    assert mu.shape[0] == logvar.shape[0]
    kl_div_list = []
    n = mu.shape[0]
    # O(N) complexity
    for i in range(1, n):
        kl_div = KL_latent(mu[i-1], logvar[i-1], mu[i], logvar[i])
        kl_div_list.append(kl_div)
    return np.array(kl_div_list)

In [389]:
def topK_KLdiv_search(kl_div_list, K=259):
    """
    kl_div_list:
    return: 
    """
    assert K >= 259
    assert len(kl_div_list) > K
    scene_change_pairs = []
    Kmax_index = kl_div_list.argsort()[-K:][::-1]
    for pair in zip(Kmax_index, Kmax_index + 1):
        scene_change_pairs.append(pair)
    return scene_change_pairs

In [390]:
def pair2image(scene_change_pairs, batch_to_image):
    """
    scene_change_pairs:
    batch_to_image: dict
    return:
    """
    scene_change_list = []
    for pair in scene_change_pairs:
        frame1 = batch_to_image[pair[0]]
        frame2 = batch_to_image[pair[1]]
        scene_change_list.append((frame1[7:], frame2[7:]))
    return scene_change_list

In [456]:
# compare predict dataframe with label dataframe
def scene_change_detect_accuracy(scene_change_labels, predict_scene_change_df):
    count = 0
    total = len(scene_change_labels)
    for i in range(len(scene_change_labels)):
        pred_frame1 = predict_scene_change_df.iloc[i]['frame1']
        pred_frame2 = predict_scene_change_df.iloc[i]['frame2']
        true_frame1_series = scene_change_labels['frame1']
        if pred_frame1 in true_frame1_series.values:
            # find corresponding index of this predict frame1 in the groud_truth df
            idx = true_frame1_series[true_frame1_series == pred_frame1].index[0]
            # compare pred_frame2 and groud truth frame2 is the same
            true_frame2 = scene_change_labels.iloc[idx]['frame2']
            if pred_frame2 == true_frame2:
                count += 1
    acc = count / total
    return acc, count

In [448]:
# read scene-change.csv
scene_change_labels = pd.read_csv("scene-change.csv")

**Float 32 Model**

In [441]:
# compute kl_divergence
kl_div_list = compute_frame_KL_div(mu, logvar)
scene_change_pairs = topK_KLdiv_search(kl_div_list)
scene_change_pairs_imagename = pair2image(scene_change_pairs, batch_to_image)

In [447]:
# Create Dataframe
predict_scene_change_df = pd.DataFrame(scene_change_pairs_imagename, columns=["frame1", "frame2"])
predict_scene_change_df = predict_scene_change_df.sort_values(by=['frame1']).reset_index(drop=True)

In [460]:
# Compute Scene change accuracy
acc, count = scene_change_detect_accuracy(scene_change_labels, predict_scene_change_df)
print("Accuracy of Float-32 Model is: {:,.2f}%".format(acc*100))
print("Correctly Identified Scene-Change: ",count)

Accuracy of Float-32 Model is: 84.17%
Correctly Identified Scene-Change:  218


**Int 8 Model**

In [444]:
# compute kl_divergence
qi_kl_div_list = compute_frame_KL_div(qi_mu, qi_logvar)
qi_scene_change_pairs = topK_KLdiv_search(qi_kl_div_list)
qi_scene_change_pairs_imagename = pair2image(qi_scene_change_pairs, batch_to_image)

In [445]:
# Create Dataframe
qi_predict_scene_change_df = pd.DataFrame(qi_scene_change_pairs_imagename, columns=["frame1", "frame2"])
qi_predict_scene_change_df = qi_predict_scene_change_df.sort_values(by=['frame1']).reset_index(drop=True)

In [461]:
# Compute Scene change accuracy
qi_acc, qi_count = scene_change_detect_accuracy(scene_change_labels, qi_predict_scene_change_df)
print("Accuracy of Int-8 Model is: {:,.2f}%".format(qi_acc*100))
print("Correctly Identified Scene-Change: ",qi_count)

Accuracy of Int-8 Model is: 84.94%
Correctly Identified Scene-Change:  220


### Visualize Predict Scene-Change and Non-Scene Change Images (5 each)

Also, what are some of the scene changes that Int-8 model captures but float-32 model doesn't?

In [455]:
def plot_images(images_tensor):
    """
    images_tensor: tensor of images -> shape(B, C, H, W)
    return: plot of images
    """
    plt.figure(figsize=(8,8))
    plt.axis("off")
    plt.imshow(np.transpose(vutils.make_grid(images_tensor, padding=2, normalize=True).cpu(),(1,2,0)))
    plt.show()

**Float 32 Model**

**Int 8 Model**